# 0. data prepare

In [1]:
import os
import shutil


def mkdir_if_not_exist(path):
    if not os.path.exists(os.path.join(*path)):
        os.makedirs(os.path.join(*path))

market_dir = '/home/hui/dataset/market1501/'
save_dir = market_dir + "/features"
force_regenerate = False
if force_regenerate or (not os.path.exists(market_dir + 'classify/')):
    classfy_dir = os.path.join(market_dir, 'classify/')
    mkdir_if_not_exist([classfy_dir])
    images_dir = market_dir + 'bounding_box_train'
    for i, image_name in enumerate(os.listdir(images_dir)):
        if image_name.split('.')[-1] != 'jpg': continue
        person_id = image_name.split('_')[0]
        mkdir_if_not_exist([classfy_dir, person_id])
        shutil.copy(os.path.join(images_dir, image_name), os.path.join(classfy_dir, person_id))
        if i % 1000==0: print i, image_name, person_id

# 1. import needed package

In [2]:
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.model_zoo import vision as model
from mxnet.gluon import nn
from mxnet.gluon.data import vision
import numpy as np
import random
import mxnet as mx
import sys
sys.path.insert(0, '../../utils')
from dataset import *
from netlib import *
import os
import shutil
from cifar10_utils import show_images
%matplotlib inline
import mxnet.gluon.model_zoo.vision as models
from tqdm import tqdm

ctx = mx.gpu(0)
num_classes = 751
def mkdir_if_not_exist(path):
    if not os.path.exists(os.path.join(*path)):
        os.makedirs(os.path.join(*path))

/root/Util/miniconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:66: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  inline backend."""
/root/Util/miniconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:71: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  'retina', 'jpeg', 'svg', 'pdf'.""")
/root/Util/miniconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:85: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  use `figure_formats` instead)""")
/root/Util/miniconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:95: DeprecationWarning: metadata {'config': True} was set from the constructor. 

# 2. data loader, data argument

In [3]:
"""
data loader
"""
batch_size = 16
# imagenet mean and std, set None or not set will use as default for image.CreateAugmenter function
mean = np.array([123.68, 116.28, 103.53])
std = np.array([58.395, 57.12, 57.375])

def _transform_test(data, label):
    im = data.astype('float32')
    im = image.imresize(im, 224, 224, interp=1) / 255
    auglist = image.CreateAugmenter(data_shape=(3, 224, 224), rand_crop=False, mean=mean, std=std)
    for aug in auglist:
        im = aug(im)
    im = nd.transpose(im, (2, 0, 1))
    return im, nd.array([label]).astype('float32')

def inv_normalize(data, clip=True):
    images = data.transpose((0, 2, 3, 1)).asnumpy()
    images = images * std + mean
    images = images.transpose((0, 3, 1, 2)) * 255
    if clip: 
        images = images.clip(0, 255)
    return images

def load_all_data_label(pathes):
    all_data, all_label = None, None
    for path in pathes:
        data, label = nd.load(path)
        label = label.reshape((-1,)).astype('float32')
        if all_data is None:
            all_data, all_label = data, label
        else:
            all_data = nd.concat(all_data, data, dim=0)
            all_label = nd.concat(all_label, label, dim=0)
    return all_data, all_label


def data_loader(batch_size, transform_train, transform_test=None, num_workers=0, pathes=None, arrayds=False):
    if transform_train is None:
        transform_train = _transform_train
    if transform_test is None:
        transform_test = _transform_test
        
    # flag=1 mean 3 channel image
    if pathes is None:
        train_ds = gluon.data.vision.datasets.ImageFolderDataset(root=market_dir + '/classify/', flag=1, transform=transform_train)
    else:
        if not arrayds:
            train_ds = MultiFolderDataset(pathes, transform=transform_train)
        else:
            train_ds = MyArrayDataset(load_all_data_label(pathes), transform=transform_train)
    #test_ds = gluon.data.vision.datasets.CIFAR10(root='~/.mxnet/datasets/cifar10', train=False, transform=transform_test)

    loader = gluon.data.DataLoader
    train_data = loader(train_ds, batch_size, shuffle=True, last_batch='keep', num_workers=num_workers)
    #test_data = loader(test_ds, batch_size, shuffle=False, last_batch='keep', num_workers=num_workers)
    return train_data, None

In [4]:
"""
data argument
"""
def transform_train_DA1(data, label):
    im = data.astype('float32')
    im = image.imresize(im, 224, 224, interp=1) / 255
    auglist = image.CreateAugmenter(data_shape=(3, 224, 224), rand_crop=False, rand_mirror=True, mean=mean, std=std)
    for aug in auglist:
        im = aug(im)
    im = nd.transpose(im, (2, 0, 1)) # channel x width x height
    return im, nd.array([label]).astype('float32')

"""
def transform_train_DA2(data, label):
    im = data.astype(np.float32) / 255
    auglist = [image.RandomSizedCropAug(size=(32, 32), min_area=0.49, ratio=(0.5, 2))]
    _aug = image.CreateAugmenter(data_shape=(3, 32, 32), resize=0, 
                                rand_crop=False, rand_resize=False, rand_mirror=True,
                                mean=np.array([0.4914, 0.4822, 0.4465]),
                                std=np.array([0.2023, 0.1994, 0.2010]),
                                brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3,
                                pca_noise=0.01, rand_gray=0, inter_method=2)
    auglist.append(image.RandomOrderAug(_aug))
    
    for aug in auglist:
        im = aug(im)
    
    im = nd.transpose(im, (2, 0, 1))
    return (im, nd.array([label]).asscalar().astype('float32'))
    

random_clip_rate = 0.3
def transform_train_DA3(data, label):
    im = data.astype(np.float32) / 255
    auglist = [image.RandomSizedCropAug(size=(32, 32), min_area=0.49, ratio=(0.5, 2))]
    _aug = image.CreateAugmenter(data_shape=(3, 32, 32), resize=0, 
                                rand_crop=False, rand_resize=False, rand_mirror=True,
#                                mean=np.array([0.4914, 0.4822, 0.4465]),
#                                std=np.array([0.2023, 0.1994, 0.2010]),
                                brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3,
                                pca_noise=0.01, rand_gray=0, inter_method=2)
    auglist.append(image.RandomOrderAug(_aug))

    for aug in auglist:
        im = aug(im)
        
    if random.random() > random_clip_rate:
        im = im.clip(0, 1)
    _aug = image.ColorNormalizeAug(mean=np.array([0.4914, 0.4822, 0.4465]),
                   std=np.array([0.2023, 0.1994, 0.2010]),)
    im = _aug(im)
    
    im = nd.transpose(im, (2, 0, 1))
    return (im, nd.array([label]).asscalar().astype('float32'))
"""
print 

# 4. define train function

In [5]:
"""
train
"""
import datetime
import utils
import sys
from random import random

num_class = 10

def abs_mean(W):
    return nd.mean(nd.abs(W)).asscalar()

def in_list(e, l):
    for i in l:
        if i == e:
            return True
    else:
        return False

def train(net, train_data, valid_data, num_epochs, lr, lr_period, 
          lr_decay, wd, ctx, w_key, output_file=None, verbose=False, loss_f=gluon.loss.SoftmaxCrossEntropyLoss(), 
          use_mixup=False, mixup_alpha=0.2, back_grad_args=None):
    
    def train_batch(data, label, i, use_backgrad, loss_f):
        data = data.as_in_context(ctx)
        _label = label.copy()
        label = label.as_in_context(ctx)
        
        # generate backgrad data
        _bloss = None
        if use_backgrad:
            args = back_grad_args
            args['prob'] = args.get('prob', 1)      # prob to use back grad data argument to prove origin data is used.
            if random() <= args['prob']:
                args['max_iters'], args['lr'] = args.get('max_iters', 60), args.get('lr', 0.1)
                _data, (_bloss,) = generate_backgrad_data(args['net'], data, label, args['max_iters'], args['lr'], iter_log=False)
                if args.has_key('soft_label_th'):
                    label = get_soft_label(label, num_class, args['soft_label_th'])
                    loss_f = args['soft_label_loss_f']
                    
                if args.has_key('show_iters') and i % args['show_iters'] == 0:
                    show_data(data[:5])
                    
                if args.has_key('attach_batch') and args['attach_batch'] == True:
                    data = nd.concat(data, _data, dim=0)
                    label = nd.concat(label, label, dim=0)
                    _label = nd.concat(_label, _label, dim=0)
                else:
                    data = _data
                    
        label = label.as_in_context(ctx)            
        with autograd.record():
            output = net(data.as_in_context(ctx))
            loss = loss_f(output, label)
        loss.backward()
        trainer.step(data.shape[0])

        _loss = nd.mean(loss).asscalar()
#         if not use_mixup and (back_grad_args is None or not args.has_key('soft_label_th')):
#             _acc = utils.accuracy(output, label)
        if not use_mixup:
            _acc = utils.accuracy(output, _label.as_in_context(ctx))
        else:
            _acc = None

        if verbose and i % 100 == 0:
            print " # iter", i,
            print "loss %.5f" % _loss, 
            if not use_mixup: print "acc %.5f" % _acc,
            print "w (",
            for k in w_key:
                w = net.collect_params()[k]
                print "%.5f, " % abs_mean(w.data()),
            print ") g (",
            for k in w_key:
                w = net.collect_params()[k]
                print "%.5f, " % abs_mean(w.grad()),
            print ")"
        return _loss, _acc, _bloss
            
    if output_file is None:
        output_file = sys.stdout
        stdout = sys.stdout
    else:
        output_file = open(output_file, "w")
        stdout = sys.stdout
        sys.stdout = output_file
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': 0.9, 'wd': wd})
    prev_time = datetime.datetime.now()
    
    if verbose:
        print " #", utils.evaluate_accuracy(valid_data, net, ctx)
    
    i = 0
    for epoch in range(num_epochs):
        train_loss = 0.
        train_acc = 0.
        bloss, btime = 0., 0
        if in_list(epoch, lr_period):
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        
        # back grad
        use_backgrad = False
        if back_grad_args is not None:
            args = back_grad_args
            if args.has_key('take_turn'):
                args['take_turn'] = not args['take_turn'] # only odd epoch(start 0) will use back grad

            if (not args.has_key('take_turn')) or args['take_turn'] == True:
                if not args.has_key('prob'): print "# back grad turn." 
                use_backgrad = True
        
        # mixup
        if not use_mixup:
            for data, label in train_data:
                _loss, _acc, _bloss = train_batch(data, label, i, use_backgrad, loss_f)
                train_loss += _loss
                if _acc is not None: train_acc += _acc
                if _bloss is not None: 
                    bloss += _bloss
                    btime += 1
                i += 1
        else:
            for x, y in train_data:
                l = x.shape[0] / 2
                data, label = mixup(x[:l], y[:l], x[l:2*l], y[l:2*l], mixup_alpha, 10)
                _loss, _, _bloss = train_batch(data, label, i, use_backgrad, loss_f)
                train_loss += _loss
                if _bloss is not None: 
                    bloss += _bloss
                    btime += 1
                i += 1
        
        # log info
        cur_time = datetime.datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        
        train_loss /= len(train_data)
        train_acc /= len(train_data)
        if train_acc < 1e-6:
            train_acc = utils.evaluate_accuracy(train_data, net, ctx)
        
        if valid_data is not None:
            valid_acc = utils.evaluate_accuracy(valid_data, net, ctx)
            epoch_str = ("epoch %d, loss %.5f, train_acc %.4f, valid_acc %.4f" 
                         % (epoch, train_loss, train_acc, valid_acc))
        else:
            epoch_str = ("epoch %d, loss %.5f, train_acc %.4f"
                        % (epoch, train_loss, train_acc))
            
        if back_grad_args is not None:
            epoch_str += (", bloss %.5f" % (-bloss / btime))
        prev_time = cur_time
        output_file.write(epoch_str + ", " + time_str + ",lr " + str(trainer.learning_rate) + "\n")
        output_file.flush()  # to disk only when flush or close
    if output_file != stdout:
        sys.stdout = stdout
        output_file.close()

# 5. get net and do EXP

In [9]:
def get_image_info(image_dir, image_name, transform=None):
    img = image.imread(os.path.join(image_dir, image_name), flag=1)
    image_name_list = image_name.split('_')
    label = int(image_name_list[0])
    cam = int(image_name_list[1].split('s')[0][1:])
    if transform is not None: 
        img, label = transform(img, label)
    return img, label, cam

def test_data_loader(image_dir, batch_size):
    batch_data, batch_label, batch_cam, batch_i = None, nd.zeros(shape=(batch_size,)), nd.zeros(shape=(batch_size,)), 0
    for image_name in os.listdir(image_dir):
        if image_name.split('.')[-1] != 'jpg': continue
        if int(image_name.split('_')[0]) == -1: continue
        image_path = os.path.join(image_dir, image_name)
        img, label, cam = get_image_info(image_dir, image_name, transform=_transform_test)
        if batch_data is None:
            batch_data = nd.zeros(shape=(batch_size, img.shape[0], img.shape[1], img.shape[2]))
        batch_data[batch_i, :, :, :] = img[:, :, :]
        batch_label[batch_i] = label
        batch_cam[batch_i] = cam
        batch_i += 1
        if batch_i == batch_size:
            yield batch_data, batch_label, batch_cam
            batch_i = 0
    if batch_i > 0:
        yield batch_data[:batch_i, :, :, :], batch_label[:batch_i], batch_cam[:batch_i]

def save_features_of_dir(net, image_dir, save_prefix):
    features, labels, cams = None, None, None
    for i, (data, label, cam) in tqdm(enumerate(test_data_loader(image_dir, batch_size=16))):
        feature = net(data.as_in_context(ctx))
        feature = feature.reshape((feature.shape[0], -1))
        if features is None:
            features, labels, cams = feature.copy(), label.copy(), cam.copy()
        else:
            features = nd.concat(features, feature, dim=0)
            labels = nd.concat(labels, label, dim=0)
            cams = nd.concat(cams, cam, dim=0)
    print "process all %d images over" % features.shape[0]
    np.save(save_prefix + 'features', features.asnumpy())
    np.save(save_prefix + 'labels', labels.asnumpy())
    np.save(save_prefix + 'cams', cams.asnumpy())
    print "all done."
    

from reid_evaluate import evaluate
from sklearn.preprocessing import normalize

def cal_CMC(prefix):
    query_feature = normalize(np.load(prefix + 'q_features.npy'), 'l2', axis=1)
    query_cam = np.load(prefix + 'q_cams.npy')
    query_label = np.load(prefix + 'q_labels.npy')
    gallery_feature = normalize(np.load(prefix + 'g_features.npy'), 'l2', axis=1)
    gallery_cam = np.load(prefix + 'g_cams.npy')
    gallery_label = np.load(prefix + 'g_labels.npy')

    CMC = np.zeros(shape=(len(gallery_label),)) # torch.IntTensor(len(gallery_label)).zero_()
    ap = 0.0
    #print(query_label)
    for i in tqdm(range(len(query_label))):
        ap_tmp, CMC_tmp = evaluate(query_feature[i],query_label[i],query_cam[i],gallery_feature,gallery_label,gallery_cam)
        if CMC_tmp[0]==-1:
            continue
        CMC = CMC + CMC_tmp
        ap += ap_tmp
        #print(i, CMC_tmp[0])

    CMC = CMC/len(query_label) #average CMC
    print('top1:%f top5:%f top10:%f mAP:%f'%(CMC[0],CMC[4],CMC[9],ap/len(query_label)))
    

## 5.1 baseline: general lr policy train my resnet50

In [9]:
def get_resnet50_v1(num_classes, pretrained=True):
    net = nn.HybridSequential()
    with net.name_scope():
        resnet = models.resnet152_v1(pretrained=pretrained, ctx=ctx)
        classify = nn.HybridSequential()
        with classify.name_scope():
            classify.add(nn.Dense(1024)) # fc6
            classify.add(nn.BatchNorm())
            classify.add(nn.Dropout(0.5))
            classify.add(nn.Activation(activation='relu'))
            classify.add(nn.Dense(num_classes)) # fc7
        net.add(resnet.features)
        net.add(classify)

    net.hybridize()
    if pretrained:
        classify.initialize(ctx=ctx)
    else:
        net.initialize(ctx=ctx)
    return net

In [10]:
num_epochs = 70
learning_rate = 0.001
weight_decay = 5e-4
lr_period = [25, 50]
lr_decay=0.1
log_file = None
loss_f = gluon.loss.SoftmaxCrossEntropyLoss()

train_data, valid_data = data_loader(batch_size, transform_train_DA1, num_workers=0)
net = get_resnet50_v1(num_classes=num_classes, pretrained=True)
net.hybridize()
train(net, train_data, valid_data, num_epochs, learning_rate, lr_period, lr_decay, weight_decay, ctx, 
      [], log_file, False, loss_f)

net.save_params("../../models/market_resnet50_v1_70e")

epoch 0, loss 6.34605, train_acc 0.0284, Time 00:13:17,lr 0.001
epoch 1, loss 4.97933, train_acc 0.1042, Time 00:13:12,lr 0.001
epoch 2, loss 3.99680, train_acc 0.2145, Time 00:13:16,lr 0.001
epoch 3, loss 3.16246, train_acc 0.3435, Time 00:13:15,lr 0.001
epoch 4, loss 2.42953, train_acc 0.4753, Time 00:13:10,lr 0.001
epoch 5, loss 1.82129, train_acc 0.6095, Time 00:13:13,lr 0.001
epoch 6, loss 1.33489, train_acc 0.7177, Time 00:13:06,lr 0.001
epoch 7, loss 0.97611, train_acc 0.8042, Time 00:13:11,lr 0.001
epoch 8, loss 0.69738, train_acc 0.8702, Time 00:13:11,lr 0.001
epoch 9, loss 0.50405, train_acc 0.9122, Time 00:13:10,lr 0.001
epoch 10, loss 0.35792, train_acc 0.9434, Time 00:13:10,lr 0.001
epoch 11, loss 0.25563, train_acc 0.9658, Time 00:13:10,lr 0.001
epoch 12, loss 0.19256, train_acc 0.9784, Time 00:13:11,lr 0.001
epoch 13, loss 0.14121, train_acc 0.9862, Time 00:13:10,lr 0.001
epoch 14, loss 0.10664, train_acc 0.9913, Time 00:13:08,lr 0.001
epoch 15, loss 0.08500, train_acc 0

In [11]:
mkdir_if_not_exist([save_dir])
net = get_resnet50_v1(num_classes=num_classes, pretrained=True)
net.hybridize()
net.load_params("../../models/market_resnet50_v1_70e", ctx=ctx)
save_features_of_dir(net[0], os.path.join(market_dir, 'bounding_box_test'), save_dir + "/resnet50_g_")
save_features_of_dir(net[0], os.path.join(market_dir, 'query'), save_dir + "/resnet50_q_")

995it [15:42,  1.06it/s]


process all 15913 images over


0it [00:00, ?it/s]

all done.


211it [02:37,  1.34it/s]


process all 3368 images over
all done.


In [10]:
cal_CMC(save_dir + "/resnet50_")

100%|██████████| 3368/3368 [00:47<00:00, 70.40it/s]


top1:0.715855 top5:0.871140 top10:0.915974 mAP:0.447167


## 5.2 baseline2

visit https://github.com/layumi/Person_reID_baseline_pytorch/blob/master/model.py

In [7]:
from mxnet.gluon import nn
class ft_net(nn.HybridBlock):
    def __init__(self, class_num, **kargs):
        super(ft_net, self).__init__(**kargs)
        resnet = model.resnet50_v1(pretrained=True)
        resnet.features

In [11]:
resnet = model.resnet50_v1(pretrained=True)

Model file is not found. Downloading.


In [9]:
def get_resnet50_v1_2(num_classes, pretrained=True):
    net = nn.HybridSequential()
    with net.name_scope():
        resnet = models.resnet152_v1(pretrained=pretrained, ctx=ctx)
        classify = nn.HybridSequential()
        with classify.name_scope():
            classify.add(nn.Dense(num_classes)) # fc7
        net.add(resnet.features)
        net.add(classify)

    net.hybridize()
    if pretrained:
        classify.initialize(ctx=ctx)
    else:
        net.initialize(ctx=ctx)
    return net

In [ ]:
num_epochs = 70
learning_rate = 0.001
weight_decay = 5e-4
lr_period = [25, 50]
lr_decay=0.1
log_file = None
loss_f = gluon.loss.SoftmaxCrossEntropyLoss()

train_data, valid_data = data_loader(batch_size, transform_train_DA1, num_workers=0)
net = get_resnet50_v1_2(num_classes=num_classes, pretrained=True)
net.hybridize()
train(net, train_data, valid_data, num_epochs, learning_rate, lr_period, lr_decay, weight_decay, ctx, 
      [], log_file, False, loss_f)

net.save_params("../../models/market_resnet50_v1_2_70e")